In [1]:
import tensorflow as tf

In [2]:
vocab_size=6110
embedding_size=128
hidden_size=128
batch_size=64
num_layers=2

In [3]:
encoder_inputs = tf.placeholder(tf.int32, [batch_size, None], name='encoder_inputs')
encoder_inputs_length = tf.placeholder(tf.int32, [None], name='encoder_inputs_length')
decoder_targets = tf.placeholder(tf.int32, [batch_size, None], name='decoder_targets')

In [4]:
embeddings = tf.Variable(tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0), dtype=tf.float32)
encoder_inputs_embedded = tf.nn.embedding_lookup(embeddings, encoder_inputs)

In [5]:
encoder_cell = tf.contrib.rnn.BasicLSTMCell(hidden_size)

In [6]:
((encoder_fw_outputs,
  encoder_bw_outputs),
 (encoder_fw_final_state,
  encoder_bw_final_state)) = (
    tf.nn.bidirectional_dynamic_rnn(cell_fw=encoder_cell,
                                    cell_bw=encoder_cell,
                                    inputs=encoder_inputs_embedded,
                                    sequence_length=encoder_inputs_length,
                                    dtype=tf.float32)
    )

In [7]:
encoder_fw_outputs

<tf.Tensor 'bidirectional_rnn/fw/fw/transpose:0' shape=(64, ?, 128) dtype=float32>

In [8]:
encoder_bw_outputs

<tf.Tensor 'ReverseSequence:0' shape=(64, ?, 128) dtype=float32>

In [9]:
encoder_fw_final_state

LSTMStateTuple(c=<tf.Tensor 'bidirectional_rnn/fw/fw/while/Exit_2:0' shape=(?, 128) dtype=float32>, h=<tf.Tensor 'bidirectional_rnn/fw/fw/while/Exit_3:0' shape=(?, 128) dtype=float32>)

In [10]:
encoder_bw_final_state

LSTMStateTuple(c=<tf.Tensor 'bidirectional_rnn/bw/bw/while/Exit_2:0' shape=(?, 128) dtype=float32>, h=<tf.Tensor 'bidirectional_rnn/bw/bw/while/Exit_3:0' shape=(?, 128) dtype=float32>)

In [11]:
encoder_outputs = tf.concat((encoder_fw_outputs, encoder_bw_outputs), 2)
encoder_final_state_c = tf.concat((encoder_fw_final_state.c, encoder_bw_final_state.c), 1)
encoder_final_state_h = tf.concat((encoder_fw_final_state.h, encoder_bw_final_state.h), 1)
encoder_final_state = tf.contrib.rnn.LSTMStateTuple(c=encoder_final_state_c, h=encoder_final_state_h)

In [12]:
decoder_cell = tf.contrib.rnn.BasicLSTMCell(hidden_size*2)

In [13]:
_, encoder_max_time = tf.unstack(tf.shape(encoder_inputs))

In [14]:
decoder_lengths = encoder_inputs_length - 1

In [15]:
softmax_w = tf.Variable(tf.random_uniform([hidden_size*2, vocab_size], -1, 1), dtype=tf.float32)
softmax_b = tf.Variable(tf.zeros([vocab_size]), dtype=tf.float32)

In [16]:
encoder_final_state

LSTMStateTuple(c=<tf.Tensor 'concat_1:0' shape=(?, 256) dtype=float32>, h=<tf.Tensor 'concat_2:0' shape=(?, 256) dtype=float32>)